# Поиск совпадающих текстов проектов НПА на `regulation.gov.ru` и на `sozd.duma.gov.ru`

В этом ноутбуке показывается, как можно делать матчинг похожих терминологически текстов. Алгоритм, описанный здесь, использовался для поиска совпадений среди текстов законопроектов в докладе ЦПУР «Качество проведения оценки регулирующего воздействия в России: что показывает сплошной анализ текстовых данных?»

## Установка зависимостей и инициализация

In [ ]:
# Пакеты для работы с текстами
%pip install pymystem3
%pip install pyaspeller
from pymystem3 import Mystem
from pyaspeller import YandexSpeller

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Для подсчета евклидова расстояния
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

# Для параллельной работы в таблицах
import pandas as pd
%pip install pandarallel
import pandarallel

## Функции для препроцессинга текстов

Наши тексты получены с помощью распознавания образов, поэтому на всякий случай можно попробовать удалить артефакты с помощью спеллера.

In [ ]:
speller = YandexSpeller()
def preprocess(text):
    try:
        corrected = speller.spelled(text)
        return corrected
    except Exception as e:
        print(e)
        return text

Следующий элемент обработки – разделение текста на токены, то есть самостоятельные кусочки: слова, знаки препинания, другие символы.
Из-за того, что что у одного слова может быть много словоформ, а также потому, что в текстах НПА могут встречаться бесполезные короткие символы (слеши, переносы строк, № и другое), необходимо также выбросить мусор и провести лемматизацию – то есть отрезание флексии слова от неизменяемой основы. 
Всё это поможет сделать размерность итогового векторного пространства меньше.

In [ ]:
stemmer = Mystem()

def get_lemma(word, coerce_punkt=True):
    if coerce_punkt:
        blank = ''
    else:
        blank = word
    stemmer_result = stemmer.analyze(word)
    if not len(stemmer_result): 
        return blank
    if 'analysis' not in stemmer_result[0]:
        return blank
    if not len(stemmer_result[0]['analysis']):
        return blank
    else:
        return stemmer_result[0]['analysis'][0]['lex']

In [ ]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmas = map(get_lemma, tokens)
    return tuple(filter(lambda x: len(x), lemmas))

## Векторизуем нужные тексты

Нам нужно векторизовать тексты, но не все: для `regulation.gov.ru` – только те, которые описывают __проекты законов__, а для `sozd.duma.gov.ru` – только те, которые были внесены не ранее начала введения ОРВ и были внесены Правительством (либо авторство неизвестно). Списки идентификаторов подходящих проектов собраны в таблицах `data/regulation_blanks.csv` и `data/duma_blanks.csv`.

In [ ]:
task_reg = pd.read_csv('data/regulation_blanks.csv', sep=';')
task_sozd = pd.read_csv('data/duma_blanks.csv', sep=';')

# Форматируем время

task_reg['public_discussion_end'] = task_reg['public_discussion_end'].apply(pd.to_datetime, errors='coerce')
task_sozd['introduction_date'] = task_sozd['introduction_date'].apply(pd.to_datetime, errors='coerce')

In [ ]:
# Скачаем из каталога ИНИД нужные датасеты 

import zipfile
import urllib.request

data_dir = 'data/'
    
url = '''
https://ds1.data-in.ru/INSERT_LINK_HERE!'''
with urllib.request.urlopen(url) as response, open(archive, 'wb') as out_file:
    data = response.read() 
    out_file.write(data)  

with zipfile.ZipFile(archive, 'r') as zip_ref:
    zip_ref.extractall(data_dir)

In [ ]:
# Импортируем тексты 

reg_texts = pd.read_csv('data/regulation_texts.csv', sep=';')
sozd_texts = pd.read_csv('data/duma_texts.csv', sep=';')

In [ ]:
# Выберем правильные тексты

reg_texts = reg_texts[reg_texts.regulation_project_id.astype(str).isin(
    task_reg.regulation_project_id.astype(str).to_list())]
sozd_texts = sozd_texts[sozd_texts.duma_project_id.astype(str).isin(
    task_sozd.duma_project_id.astype(str).to_list())]

# Соберем в один корпус (так удобнее),
# но сохраним количество текстов с regulation,
# чтобы потом их разделить

full_corpus = reg_texts.text.to_list() + sozd_texts.text.to_list()
full_reg_num = len(reg_texts)

In [ ]:
# Векторизуем – это займёт долгое время!

vectorizer = TfidfVectorizer(
    ngram_range=(2, 4), 
    preprocessor=preprocess, 
    tokenizer=tokenize, 
    min_df=2, 
    max_df=0.2)

vectors = vectorizer.fit_transform(full_corpus)

In [ ]:
# Разделим назад

vectors_reg = [x for x in vectors[:full_reg_num]]
vectors_sozd = [x for x in vectors[full_reg_num:]]

In [ ]:
# Хэшируем вектора, чтобы быстро их искать

reg_text_lookup = dict(zip(reg_texts.filename.to_list(), vectors_reg))
sozd_text_lookup = dict(zip(sozd_texts.filename.to_list(), 
                            [x.toarray() for x in vectors_sozd]))

# Запишем в табличку
task_reg['vector'] = task_reg.regulation_id.map(reg_text_lookup)

# Выбросим наблюдения с неудачной векторизацией
task_reg = task_reg[~task_reg.vector.isna()]
task_sozd = task_sozd[task_sozd.gosduma_id.isin(sozd_text_lookup)]

## Поиск ближайшего текста

Это – самая трудоёмкая процедура, которая может занять несколько суток даже на большом количестве ядер.
Чтобы облегчить задачу, вы можете сузить пространство поиска (например, приняв во внимание название или считая, что документ должен оказаться в Думе не позже, чем спустя год после того, как прошел ОРВ – у нас по умолчанию два года).

Также при желании можно возвращать не только лучший матч, но и первые несколько лучших.

In [ ]:
# Поиск ближайшего для наблюдения из регулейшена

def find_best(row):
    reg_id = row.regulation_id
    reg_date = row.enddiscussion
    reg_vector = row.vector.toarray()
    
    row['similarity'] = np.nan
    
    if not pd.isnull(reg_date):
        possible_sozd = task_sozd[(task_sozd.gosduma_date >= reg_date) &
                                 (task_sozd.gosduma_date.apply(lambda x: x.year - reg_date.year <= 2)) &
                                  (task_sozd.gosduma_id.isin(sozd_text_lookup))].copy()
    else:
        possible_sozd = task_sozd.copy()
        
    possible_ids = possible_sozd.gosduma_id.to_list()
    if len(possible_ids) < 1:
        return row
    
    def calculate_similarity_by_id(gosduma_id):
        sozd_vector = sozd_text_lookup[gosduma_id]
        return float(cosine_similarity(reg_vector, sozd_vector))
                
    pandarallel.initialize()  
    possible_sozd['similarity'] = possible_sozd.gosduma_id.parallel_apply(calculate_similarity_by_id)

    if len(possible_sozd) > 0:
        best = possible_sozd.sort_values('similarity', ascending=False).reset_index().loc[0]
        row['gosuma_id'] = best.gosduma_id
        row['similarity'] = best.similarity
        row['gosduma_date'] = best.gosduma_date
        row['gosduma_status'] = best.gosduma_status
        row['gosduma_stage'] = best.gosduma_stage
        row['gosduma_solution'] = best.gosduma_solution
        
    return row

In [ ]:
# Запускаем!
best_matches = task_reg.apply(find_best, axis=1)

## Поиск порогового значения

Найдём пороговое значение, начиная с которого тексты будем считать совпадающими.
Это необязательно `1.0` – формат текстов может отличаться, а также тексты могут серьёзно редактироваться.

В докладе мы брали `0.75`-перцентиль похожести среди известных нам, то есть около `0.6`

In [ ]:
# Известные матчи и не-матчи
known_matches = pd.read_csv('data/known_matches.csv')

# Интересные для теста id
train_reg_ids = set(
    known_matches.regulation_id.to_list())
train_sozd_ids = set(
    known_matches.gosduma_id.to_list())

# Интересные для теста тексты
reg_texts_train = reg_texts[reg_texts.filename.isin(train_reg_ids)].copy()
sozd_texts_train = sozd_texts[sozd_texts.filename.isin(train_sozd_ids)].copy()

In [ ]:
# Векторизуем только эти тексты
corpus = reg_texts_train.text.to_list() + sozd_texts_train.text.to_list()
reg_num = len(reg_texts_train.text.to_list())
train_vectors = vectorizer.transform(corpus)

In [ ]:
# Разделим назад
reg_texts_train['vectors'] = [x.toarray() for x in train_vectors[:reg_num]]
sozd_texts_train['vectors'] = [x.toarray() for x in train_vectors[reg_num:]]

In [ ]:
# Подготовимся искать
reg_lookup = dict(zip(reg_texts_train.filename.to_list(), reg_texts_train['vectors']))
sozd_lookup = dict(zip(sozd_texts_train.filename.to_list(), sozd_texts_train['vectors']))

In [ ]:
# Подсчитаем расстояние для двух известных векторов
import numpy as np

def calculate_similarity(row):
    reg_id = row.regulation_id
    sozd_id = row.gosduma_id
    try:
        x = reg_lookup.get(reg_id)
        y = sozd_lookup.get(sozd_id)
        return float(cosine_similarity(x, y))
    except Exception as e:
        pass
    
known_matches['similarity'] = known_matches.apply(
    calculate_similarity, axis=1)

In [ ]:
# Посмотрим описательные статистики
known_matches[~known_matches.similarity.isna()].similarity.describe()

In [ ]:
# Выберем пороговое значение
threshold = 0.6

# Соберём все достаточно совпадающие пары
matches = best_matches[best_matches.similarity.apply(lambda x: not pd.isna(x) and x >= threshold)]
matches.drop('vector', axis=1, inplace=True)
matches.to_csv('data/matches.csv', sep=';', index=False)